In [1]:
import SimpleITK as sitk
import pandas as pd
import numpy as np
import napari
import vispy
import tqdm
import gzip
import time
import cv2

from napari.utils.colormaps import colormap_utils as cu
from pathlib import Path

c:\Users\zoeba\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Stratégie globale:

Après avoir récupéré les masques générés par network_construction, l'idée est d'extraire des paramètres des ellipses qui correspondent à la géométrie extraite. Ensuite, on détecte les ellipses anormales grâce à un écart à un modèle linéaire fitté sur chacun des paramètres, et on réadapte leurs paramètres. On dilate alors les ellipses pour bien sélectionner tous les pixels d'intérêt. 

On sélectionne les pixels à l'intérieur des ellipses, ces zones sont censées correspondre aux vaisseaux, on peut donc les étudier facilement:
- les calcifications sont extraites avec un seuil sur les niveaux de blancs.
- la distribution des niveaux de gris est étudiée pour les occulusions et sténoses.


### Création des ellipses 

In [2]:
def find_slice_with_mask(mask):
    '''
    Détecte les index de la première et dernière coupes d'un masque
    Args:
        mask (numpy array): le masque ou le fond est des 1 et le masque des 0
    Return:
        (rangemin, rangemax) (int,int): tuple avec l'index des première et dernière coupes d'un masque. 
    '''
    nb_slices_with_masks = []
    for nb_slice in range(mask.shape[0]-1, 0,-1):
        if 255 in mask[nb_slice]:
            nb_slices_with_masks.append(nb_slice)
    rangemin = np.min(nb_slices_with_masks)
    rangemax = np.max(nb_slices_with_masks)
    return (rangemin, rangemax)

In [3]:
def fit_ellipses(mask, rangemin, rangemax):
    ''' 
    Le but est de récuperer les paramètres d'une liste d'ellipses à partir de la liste de masques fournis par le network construction
    Args :
        mask (numpy array) : masque des artères
        rangemin (int): début des coupes sélectionnées
        rangemax (int): fin des coupes sélectionnées
    Return:
        list_params_ellipses (list[list[double]]): liste contenant les paramètres de chaque ellipse
    '''
    mean_nb_pixel=np.sum(mask) / ((rangemax-rangemin) * 255)

    if mean_nb_pixel > 6:    
        list_params_ellipses=[]
        for index_img in (reversed(range(rangemax-rangemin))):
            img=mask[rangemin+index_img]
            contours_found,_=cv2.findContours(img.astype(np.uint8), method=cv2.RETR_LIST, mode=cv2.CHAIN_APPROX_SIMPLE)
            if len(contours_found)>0: 
                if len(contours_found[0])>5:
                    ellipse=cv2.fitEllipse(contours_found[0])
                    ellipse_ok=((ellipse[0][0], ellipse[0][1]), (ellipse[1][0]/2,ellipse[1][1]/2), ellipse[2])
                    list_params_ellipses.append(ellipse_ok)
                else:
                    try:
                        ellipse_ok=((list_params_ellipses[-1][0][0], list_params_ellipses[-1][0][1]), (list_params_ellipses[-1][1][0]/2,list_params_ellipses[-1][1][1]/2), list_params_ellipses[-1][2])
                        list_params_ellipses.append(ellipse_ok)
                    except IndexError:
                        ellipse_ok = ((contours_found[0][0][0][0], contours_found[0][0][0][1]), (1,1), 0)
                        list_params_ellipses.append(ellipse_ok)
            else:
                print("erreur nb contours")
    else:
        return None
                
    list_params_ellipses.reverse()
    return list_params_ellipses

In [4]:
def predict_and_compare_values(list_params, rangemin, all_ranges_max):
    '''
    Ajuste les paramètres des ellipses les unes par rapport aux autres. 
    Args:
        list_params (list): les paramètres originaux des ellipses pour un masque donné
        rangemin (int) : l'index de la première coupe du masque
        all_ranges_max (list[int]): les index maximums de tous les masques
    Return:
        new_list_params (list): les paramètres des ellipses après modification pour un masque donné
        list_products_ab (list[double]) : valeur du produit a*b pour chaque ellipse du masque
        list_products_ab_predicted (list[double]) : valeur du produit a*b d'après la regression linéaire
    '''
    
    new_list_params = [list_params[-1]]
    list_products_ab = [my_tuple[1][0]*my_tuple[1][1] for my_tuple in list_params]
    list_indexes = [ i for i in range(len(list_products_ab))]
    #regression linéaire de degré 1 sur le produit a*b, grand et petit axe
    fit = np.polyfit(list_indexes, list_products_ab, deg=1)
    predict_size_area = np.poly1d(fit)
    list_products_ab_predicted = [predict_size_area(index) for index in list_indexes]
    
    for index in reversed(range(len(list_products_ab_predicted)-1)):
        product_predicted=list_products_ab_predicted[index]
        if np.abs((product_predicted-list_products_ab[index])*100/product_predicted)<80: #si moins de 80 pourcent d'erreurs, on garde valeur originale
            new_list_params.append(list_params[index])
        elif (index+rangemin) in all_ranges_max:#si on est à une jonction de 2 masques = bifurcation, on garde les paramètres initiaux
            new_list_params.append(list_params[index])
        else: #sinon, on met les paramètres de l'ellipse précédente
            new_list_params.append(new_list_params[-1])
    new_list_params.reverse()

    return new_list_params, list_products_ab, list_products_ab_predicted
    

In [5]:
def dilation(img_3D, kernel_size):
    ''' 
    Dilatation d'une image 
    Args:
        img_3D (numpy array): image à dilater
        kernel_size (int): taille du kernel
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    for index, slice in enumerate(img_3D):
        img_3D[index] = cv2.dilate(img_3D[index] , kernel, iterations=1)

In [6]:
def create_3d_ellipses(list_params_ellipses, shape_image, rangemin, rangemax, multiple_values = False, list_values = [], dilate=False):
    ''' 
    Trace des ellipses sur une image 3D blanche
    Args:
        list_params_ellipses (list[list[double]]): liste des paramètres de chaque ellipse
        shape_image (tuple): taille de l'image 3D que l'on veut créer
        rangemin (int): première coupe du masque
        rangemax (int): dernière coupe du masque
        multiple_values (boolean): si la valeur des pixels utilisés pour remplir les ellipses sont les mêmes sur toutes les coupes
        list_values (list[int]): liste indiquant la valeur de pixel à utiliser pour chaque coupe
        dilate (Boolean): indicateur de si l'image doit être dilatée ou non
    Return:
        ellipses_3d (numpy array): image 3D avec les ellipses dessinées dessus
    '''
    ellipses_3d = []
    for index_img in (range(rangemax-rangemin)):

        if multiple_values:
            value = list_values[index_img] #valeur de pixel à utiliser pour l'ellipse de cette coupe
        else:
            value = 255
            
        ellipse_mask = np.zeros((shape_image[1],shape_image[2]), dtype=np.uint8) #image blanche
        params = list_params_ellipses[index_img]
        center = (int(params[0][0]),int(params[0][1]))
        axis = (int(params[1][0]),int(params[1][1]))
        angle = int(params[2])
        cv2.ellipse(ellipse_mask, center, axis, angle, 0, 360, value, -1) #dessiner l'ellipse
        ellipses_3d.append(ellipse_mask)

    numpy_ellipse_3d = np.array(ellipses_3d)
    
    if dilate:
        dilation(numpy_ellipse_3d, kernel_size = 4)

    return numpy_ellipse_3d

### Etude statistique sur les ellipses

In [7]:
def keeping_arteries_only(filled_ellipses, image):
    '''
    Le but est de garder l'image originale en la restreignant à l'intérieur des ellipses.
    Args:
        filled_ellipses (numpy array): les ellipses remplies
        image (numpy array): image d'origine 
    Return:
        list_arteries_only (numpy array): l'image originale restreinte à l'intérieur des ellipses
    ''' 
    list_arteries_only = np.where(filled_ellipses == 255, image, 0)
    return list_arteries_only

In [8]:
class Descriptor:
    '''  
    Cette classe contient toutes les propriétés intéressantes d'une coupe d'une image.

    Attributes:
        index (int) : index de la coupe dont on a les propriétés
        list_pixels (list[int]) : liste de tous les pixels de la coupe
        list_pixels_without_calcifications (list[int]) : liste de tous les pixels après avoir enlevé les calcifications
        mean (double) : moyenne des pixels sans calcifications de la coupe
        sd (double) : écart-type des pixels sans calcification de la coupe
        proportion_calcification (double) : le pourcentage de calcification dans la coupe
        ab (double) : produit des paramètres a et b de l'ellipse de cette coupe
        ab_pred (double) : produit des paramètres a et b de l'ellipse prédite
        list_params_ellipse (list[tuple]) : paramètres de l'ellipse 
        deviation_mean (double) : écart relatif en pourcentage entre la moyenne de cette coupe et la moyenne de toutes les coupes du masque
        deviation_sd (double) : écart relatif en pourcentage entre l'écart-type de cette coupe et la moyenne des écarts-types de toutes les coupes du masque
        Z (double) : Zscore

    Methods:
        __init__(self, index, list_pixels, ab, ab_pred, list_params_ellipse)
            Initialise une nouvelle instance de la classe
        
        remove_calcifications(self, list_pixels)
            Enlève les pixels de valeur supérieure à 250 qui sont considérés comme des calcifications

        compute_mean(self)
            Calcule la moyenne de la liste des pixels sans calcifications

        compute_sd(self)
            Calcule l'écart-type de la liste des pixels sans calcifications

        compute_deviation_mean(self, mean_mean_slice_mask)
            Calcule l'écart relatif entre la moyenne de la coupe et mean_mean_slice_mask

        compute_deviation_sd(self, mean_sd_slice_mask)
            Calcule l'écart relatif entre l'écart-type de la coupe et mean_sd_slice_mask

        compute_z(self, mean_mean_slice_mask, mean_sd_slice_mask)
            Calcule le Z score

        compute_calcification(self)
            Calcule le pourcentage de calcifications
    '''

    def __init__(self, index, list_pixels, ab, ab_pred, list_params_ellipse):
        '''  
        Initialise une nouvelle instance de la classe
        Args:
            index : index de la coupe
            list_pixels : liste des pixels dans la coupe
            ab : produit des paramètres a et b de l'ellipse 
            ab_pred : produit prédit des paramètres a et b de l'ellipse
            list_params_ellipse : liste de tous les paramètres de l'ellipse
        '''
        self.index = index
        self.list_pixels = list_pixels
        self.list_pixels_without_calcifications = self.remove_calcifications(list_pixels)
        self.mean = self.compute_mean()
        self.sd = self.compute_sd()
        self.proportion_calcification = self.compute_calcification()
        self.ab = ab
        self.ab_pred = ab_pred
        self.list_params_ellipse = list_params_ellipse

        self.deviation_mean = -1
        self.deviation_sd = -1
        self.z = -1

    def get_index(self):
        return  self.index

    def get_list_pixels(self):
        return  self.list_pixels

    def get_list_pixels_without_calcifications(self):
        return  self.list_pixels_without_calcifications
     
    def get_mean(self):
        return self.mean
    
    def get_sd(self):
        return self.sd

    def get_proportion_calcification(self):
        return self.proportion_calcification
    
    def get_ab(self):
        return self.ab

    def get_ab_pred(self):
        return self.ab_pred
    
    def get_list_params_ellipse(self):
        return self.list_params_ellipse
    
    def get_deviation_mean(self):
        return self.deviation_mean
    
    def get_deviation_sd(self):
        return self.deviation_sd
    
    def get_z(self):
        return self.z

    def remove_calcifications(self, list_pixels):
        '''
        Enlève les pixels de valeur supérieure à 250 qui sont considérés comme des calcifications
        Args :
            list_pixels (list[int]): liste des pixels de la coupe
        Return :
            list_without_calcifications (list[int]): liste des pixels sans les calcifications
        '''
        list_without_calcifications = [valeur for valeur in list_pixels if valeur < 250]
        return list_without_calcifications

    def compute_mean(self):
        '''
        Calcule la moyenne de la liste des pixels sans calcifications
        Return :
            (double) : la moyenne de la coupe
        '''
        return np.mean(self.list_pixels_without_calcifications)

    def compute_sd(self):
        '''
        Calcule l'écart-type de la liste des pixels sans calcifications
        Return :
            (double) : l'écart-type de la coupe
        '''
        return np.std(self.list_pixels_without_calcifications)
    
    def compute_deviation_mean(self, mean_mean_slice_mask):
        '''
        Calcule l'écart relatif entre la moyenne de la coupe et mean_mean_slice_mask
        Args : 
            mean_mean_slice_mask (double) : la valeur avec laquelle on veut comparer la moyenne de la coupe
        '''
        self.deviation_mean = ((self.mean-mean_mean_slice_mask)/mean_mean_slice_mask)*100
    
    def compute_deviation_sd(self, mean_sd_slice_mask):
        '''
        Calcule l'écart relatif entre l'écart-type de la coupe et mean_sd_slice_mask
        Args : 
            mean_sd_slice_mask (double) : la valeur avec laquelle on veut comparer l'écart-type de la coupe
        '''
        self.deviation_sd = ((self.sd-mean_sd_slice_mask)/mean_sd_slice_mask)*100

    def compute_z(self, mean_mean_slice_mask, mean_sd_slice_mask):
        '''
        Calcule le Zscore
        Args :
            mean_mean_slice_mask (double) : valeur à comparer avec la moyenne
            mean_sd_slice_mask (double) : valeur à comparer avec l'écart-type
        '''
        self.z = (self.mean - mean_mean_slice_mask)/np.sqrt(self.sd + mean_sd_slice_mask)

            
    def compute_calcification(self):
        '''
        Calcule le pourcentage de calcifications
        Return:
            proportion (double) : pourcentage de calcification
        '''
        nb_pixel_250 = sum(1 for valeur in self.list_pixels if valeur > 250)
        proportion = nb_pixel_250 / len(self.list_pixels)
        return proportion

In [9]:
def compute_stats(info_one_mask, arteries):  
    '''
    Créer une liste d'objets Descriptor pour chaque coupe d'un masque
    Args:
        info_one_mask(dic): dictionnaire avec les informations du masque
        arteries(numpy array) : image 3D
    ''' 
    rangemin = info_one_mask['rangemin']
    rangemax = info_one_mask['rangemax']
    arteries = arteries[rangemin:rangemax]
    ellipses3D = info_one_mask['ellipses_3D'][rangemin:rangemax]
    list_ab = info_one_mask['list_ab']
    list_ab_predicted = info_one_mask['list_ab_predicted']
    list_param_ellipses = info_one_mask['list_params_ellipses']

    list_descriptor = []
    #Récupérer une liste de valeurs de pixels par slice, en prenant en compte uniquement les pixels dans l'ellipse
    arteries_only = np.where(ellipses3D == 255, arteries, np.nan)

    # créer l'objet descriptor pour chaque slice
    for index, slice in enumerate(arteries_only):
        slice = slice.flatten()
        slice = list(slice[np.logical_not(np.isnan(slice))]) # remove nan
        list_descriptor.append(Descriptor(index+rangemin, slice, list_ab[index], list_ab_predicted[index], list_param_ellipses[index]))
    
    list_reference = [] # Tous les pixels du masque
    list_mean_descriptors = []
    list_sd_descriptors = []
    for descriptor in list_descriptor:
        list_reference = np.concatenate((list_reference, descriptor.get_list_pixels_without_calcifications()))
        list_mean_descriptors.append(descriptor.get_mean())
        list_sd_descriptors.append(descriptor.get_sd())

    for descriptor in list_descriptor:
        descriptor.compute_deviation_mean(np.mean(list_mean_descriptors))
        descriptor.compute_deviation_sd(np.mean(list_sd_descriptors))
        descriptor.compute_z(np.mean(list_mean_descriptors), np.mean(list_sd_descriptors))

    return list_descriptor

In [10]:
def convert_stats_to_table(list_descriptor):
    '''  
    Créer un tableau regroupant les caractéristiques des objets
    Args:
        list_descriptor (list[Descriptor]) : liste des objets descriptor de chaque coupe
    Return:
        df (pd.DataFrame) : tableau des caractéristiques de toutes les coupes d'un masque
    '''
    tableau_attributs = [
        [int(objet.get_index()), 
         objet.get_mean(), 
         objet.get_deviation_mean(), 
         objet.get_sd(), 
         objet.get_deviation_sd(), 
         objet.get_z(),
         objet.get_proportion_calcification(), 
         objet.get_ab(), 
         objet.get_ab_pred(),
         objet.get_list_params_ellipse()[0][0], #xc
         objet.get_list_params_ellipse()[0][1], #yc
         objet.get_list_params_ellipse()[1][0], #a
         objet.get_list_params_ellipse()[1][1], #b
         objet.get_list_params_ellipse()[2], #theta
         ] for objet in list_descriptor
    ]

    df = pd.DataFrame(tableau_attributs,
        columns=['index', 'mean', 'deviation_mean', 'sd', 'deviation_sd', 'z', 'proportion_calcification',
            'ab', 'ab_pred', 'xc', 'yc', 'a', 'b', 'theta'])
    
    return df

In [11]:
def create_heatmap(mask_shape, df, str_value, rangemin, rangemax):
    '''
    Créé une heatmap, c'est à dire une image 3D avec des ellipses ayant des valeurs de pixels différentes selon les coupes en fonction d'une propriété de la coupe  
    Args:
        mask_shape (liste[int]): taille de l'image 3D que l'on veut obtenir
        df (pd.Dataframe) : liste des propriétés des coupes
        str_value (str) : colonne du dataframe que l'on veut utiliser pour la heatmap
        rangemin (int): première coupe du masque
        rangemax (int): dernière coupe du masque
    Return:
        whole_range_ellipses_heatmap(numpy array) : image 3D avec des ellipses de différentes couleurs
    '''
    list_params_ellipses = []
    for _, line in df.iterrows():
        list_params_ellipses.append([(line['xc'], line['yc']),(line['a'], line['b']), line['theta']])
    #les amplitudes sont modifiées selon les paramètres à afficher
    if str_value == "proportion_calcification": 
        min_value=0.85
        max_value=1
    else:
        min_value=-3
        max_value=3

    values_heatmap = []
    # mapper la valeur de la statistique qui nous intéresse entre 0 et 255
    for _, line in df.iterrows():
        if max_value ==0:
            values_heatmap.append(0)
        else:    
            if str_value == "proportion_calcification":
                actual_value = 1-line[str_value]
                value_to_append = int(((actual_value - (min_value))*254/(max_value-min_value))+1)
                values_heatmap.append(value_to_append)
            else:
                values_heatmap.append(int(((line[str_value]+np.abs(min_value))*254/(max_value-min_value))+1))

    ellipses_heatmap = create_3d_ellipses(list_params_ellipses, mask_shape, rangemin, rangemax, multiple_values = True, list_values = values_heatmap, dilate=True)
    #retour au format 3D initial
    before = np.zeros((rangemin, mask_shape[1],mask_shape[1]))
    after = np.zeros((mask_shape[0]-rangemax, mask_shape[1], mask_shape[1]))
    whole_range_ellipses_heatmap = np.concatenate((before, ellipses_heatmap, after))

    return whole_range_ellipses_heatmap

### Construction des réseaux

Assemblage des masques dans un réseau 3D global.

In [12]:
def construct_mask_network(mask_list, min_range, max_range):
    '''
    En partant de la liste de masques, le réseau complet est construit. 
    Args:
        mask_list (list[np.array]): la liste des masques construite par le network construction
        min_range (int): l'index du premier masque à traiter
        max_range (int): l'index du dernier masque à traiter

    Return:
        mask_network (np.array): l'image 3D où le réseau de mask est reconstitué
    ''' 
    mask_network = np.zeros(mask_list[0].shape)
    print("Constructing the mask network...", end= " ")
    for index_mask in (range(min_range, max_range)):
        mask_network = np.where(mask_list[index_mask]!=0, mask_list[index_mask], mask_network)  
    print("Done")
    return mask_network

In [13]:
def construct_ellipses_network(mask, rangemin, rangemax, all_ranges_max): 
    '''
    En partant de la liste de masques construite par le network construction, des ellipses sont approximées pour reconstruire un réseau artériel lisse. 
    Args:
        mask_list (list[np.array]): la liste des masques construite par le network construction
        rangemin (int): première coupe du masque dans le bloc 3D total
        rangemax (int): dernière coupe du masque dans le bloc 3D total
        all_ranges_max (list[int]): les index maximums de tous les masques

    Return:
        info_all_masks (list[dic]): une liste où pour chaque ligne (qui correspond à un masque), il y a un dictionnaire où les clefs sont: rangemin, rangemax, mask, ellipses_3D, list_ab, list_ab_predicted, list_params_ellipses
    ''' 
    list_params = fit_ellipses(mask, rangemin, rangemax)
    if list_params == None:
        return None
       
    new_list_param, list_products_ab, list_products_ab_predicted = predict_and_compare_values(list_params, rangemin, all_ranges_max)

    new_ellipses_3d = create_3d_ellipses(new_list_param, mask.shape, rangemin, rangemax, dilate=True)
    #retour au format 3D initial
    before = np.zeros((rangemin, mask.shape[1],mask.shape[1]))
    after = np.zeros((mask.shape[0]-rangemax, mask.shape[1],mask.shape[1]))
    whole_size_ellipses_3D = np.concatenate((before, new_ellipses_3d, after))

    info_of_one_mask={'rangemin': rangemin,
                      'rangemax': rangemax,
                      'mask': mask, 
                      'ellipses_3D': whole_size_ellipses_3D, 
                      'list_ab': list_products_ab,
                      'list_ab_predicted': list_products_ab_predicted,
                      'list_params_ellipses': new_list_param
                      }
    
    return info_of_one_mask

In [14]:
def constructing_all_networks(image_preprocessed, mask_list, min_range, max_range, param_to_study="z", reconstruct_ellipse=False, reconstruct_arteries=False, reconstruct_masks=False, reconstruct_heatmap=False):
    '''
    Fonction qui permet de construire les réseaux d'artères, d'ellipses et de masks. 
    Args:
        image_preprocessed (np.array): l'image pré-traitée
        mask_list (list[np.array]): la liste des masque construite par la network construction
        min_range (int): l'index du premier masque à traiter
        max_range (int): l'index du dernier masque à traiter
        param_to_study (str): nom du paramètre à étudier
        reconstruct_ellipse (boolean): indique si le réseau d'ellipses doit être construit
        reconstruct_arteries (boolean): indique si le réseau d'artères doit être construit
        reconstruct_masks (boolean): indique si le réseau de masks doit être construit

    Return:
        mask_network (np.array): l'image 3D où le réseau de mask est reconstitué
        ellipses_network (np.array): l'image 3D où le réseau d'ellipses est reconstitué
        arteries_network (np.array): l'image 3D où le réseau d'artères est reconstitué
    ''' 
    ellipses_network = np.zeros_like(mask_list[0])
    ellipses_heatmap_network = np.zeros_like(mask_list[0])
    arteries_network = np.zeros_like(mask_list[0])
    mask_network = np.zeros_like(mask_list[0])

    list_table_stats = []

    all_ranges_max = [] #sauvegarde des index quand il y a une bifurcation
    for index in range(len(mask_list)):
        all_ranges_max.append(find_slice_with_mask(mask_list[index])[1])

    if (max_range-min_range) == 1: #si un seul masque, pas besoin de reconstruction
        print(1, " out of ", 1)
        rangemin, rangemax = find_slice_with_mask(mask_list[min_range])
        info_one_mask = construct_ellipses_network(mask_list[min_range], rangemin, rangemax+1, all_ranges_max)
        if info_one_mask != None:
            ellipses_network = info_one_mask['ellipses_3D']
            arteries_network = keeping_arteries_only(info_one_mask['ellipses_3D'], image_preprocessed)
            mask_network = mask_list[min_range]
            
            list_descriptor = compute_stats(info_one_mask, arteries_network)
            df = convert_stats_to_table(list_descriptor)
            list_table_stats.append(df)

            ellipses_heatmap_network = create_heatmap(mask_list[min_range].shape, df, param_to_study, rangemin, rangemax+1)

    else: #sinon, parcours de la liste de masques
        for index in tqdm.tqdm(range(min_range,max_range)):
            rangemin, rangemax = find_slice_with_mask(mask_list[index])
            info_one_mask = construct_ellipses_network(mask_list[index], rangemin, rangemax+1, all_ranges_max)
            if info_one_mask != None:
                if reconstruct_ellipse:
                    ellipses_network = np.where(info_one_mask['ellipses_3D']!=0, info_one_mask['ellipses_3D'], ellipses_network)
                arteries = keeping_arteries_only(info_one_mask['ellipses_3D'], image_preprocessed)
                if reconstruct_arteries:
                    arteries_network = np.where(arteries!=0, arteries, arteries_network)
                    
                list_descriptor = compute_stats(info_one_mask, arteries)
                df = convert_stats_to_table(list_descriptor)
                list_table_stats.append(df) 

                ellipses_heatmap = create_heatmap(mask_list[min_range].shape, df, param_to_study, rangemin, rangemax+1)
                if reconstruct_heatmap:
                    ellipses_heatmap_network = np.where(ellipses_heatmap!=0, ellipses_heatmap, ellipses_heatmap_network)
            
        if reconstruct_masks :
            mask_network = construct_mask_network(mask_list, min_range, max_range)
    return mask_network, ellipses_network, arteries_network, ellipses_heatmap_network, list_table_stats

# Imports et éxécution.

In [15]:
f = gzip.GzipFile('temp/mask_list2.npy.gz', "r")
mask_list = np.load(f)
print(mask_list.shape)
image_preprocessed = np.load(Path("temp/preprocessed_image2.npy"))

(13, 1530, 512, 512)


In [16]:
minrange = 0
maxrange = len(mask_list)
mask_network, ellipses_network, arteries_network, ellipses_heatmap_network, list_table_stats = constructing_all_networks(
    image_preprocessed, 
    mask_list, 
    min_range = minrange, 
    max_range = maxrange, 
    param_to_study = "z",
    reconstruct_ellipse = False, 
    reconstruct_arteries = False, 
    reconstruct_masks = True,
    reconstruct_heatmap = True)

  0%|          | 0/2 [00:00<?, ?it/s]

mean  58.790055248618785  time sum  0.5979583263397217  len mask 181


 50%|█████     | 1/2 [00:14<00:14, 14.18s/it]

mean  68.63829787234043  time sum  0.6202723979949951  len mask 47


100%|██████████| 2/2 [00:33<00:00, 16.70s/it]


Constructing the mask network... Done


Visualisation des résultats: masques et heatmap sur l'image.

In [17]:
viewer = napari.Viewer()
custom = vispy.color.Colormap([[0.0, 0.0, 0.0],[1.0, 0.0, 0.0],[1.0, 0.37, 0.12],[1.0, 1.0, 0.0], [0.0, 1.0, 0.0]],controls=(0,0.003,0.25,0.5,1.0))
image_layer = viewer.add_image(image_preprocessed, colormap="gray")
image_layer = viewer.add_image(mask_network, colormap="blue", contrast_limits=(0,255), blending="additive")
image_layer = viewer.add_image(ellipses_heatmap_network, colormap=custom, contrast_limits=(0,255), blending="additive")
napari.run()

Sauvegarde des résultats:


In [18]:
# img_ellipses_heatmap_network = sitk.GetImageFromArray(ellipses_heatmap_network)
# sitk.WriteImage(img_ellipses_heatmap_network, "temp/img_ellipses_heatmap_network2_CALCIF.mhd")

# img_image_preprocessed = sitk.GetImageFromArray(image_preprocessed)
# sitk.WriteImage(img_image_preprocessed, "temp/image_preprocessed1.mhd")

A enlever, chargement et visualisation des images sauvées.

In [19]:
# mr_image = sitk.ReadImage(Path("temp/img_ellipses_heatmap_network1.mhd"))
# heatmap = sitk.GetArrayViewFromImage(mr_image)

# mr_image2 = sitk.ReadImage(Path("temp/image_preprocessed1.mhd"))
# preprocessed = sitk.GetArrayViewFromImage(mr_image2)


In [20]:
# viewer = napari.Viewer()
# custom = vispy.color.Colormap([[0.0, 0.0, 0.0],[1.0, 0.0, 0.0],[1.0, 0.37, 0.12],[1.0, 1.0, 0.0], [0.0, 1.0, 0.0]],controls=(0,0.003,0.25,0.5,1.0))
# image_layer = viewer.add_image(preprocessed, colormap="gray")
# image_layer = viewer.add_image(heatmap, colormap=custom, contrast_limits=(0,255), blending="additive")
# napari.run()

In [21]:
# napari avec une couleur par masque
# viewer = napari.Viewer()
# image_layer = viewer.add_image(image_preprocessed, colormap="gray", contrast_limits=(0,255))
# colorindex = 0
# for mask in mask_list:
#     if np.sum(mask) > 0:
#         image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex%5],blending="additive")
#         colorindex+=1
# napari.run()